In [1]:
%load_ext autoreload
%autoreload 2
    
import os, sys

os.environ['DYLD_LIBRARY_PATH']        = os.path.join(os.environ.get('CONDA_PREFIX',''), 'lib')
os.environ['DYLD_FALLBACK_LIBRARY_PATH'] = ''
#PWD = os.getenv('PWD')
PWD = '/Users/chris/Documents/lab/emAnalysis/backend/server'

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

init_django("server")
from django.conf import settings
media_root = settings.MEDIA_ROOT
import numpy as np
import os
from matplotlib import pyplot as plt
import functools
import math
import sims
import copy
import cv2
import os
from pathlib import Path
import numpy as np
from skimage.measure import regionprops
from scipy import spatial
import pickle
from matplotlib.figure import Figure
import pyvips
from core.models import Canvas
from mims.models import MIMSImage, MIMSImageSet
from scipy.ndimage import rotate
import pprint
import sims
import SimpleITK as sitk
import tifffile

In [5]:
from mims.models import MIMSImage
from pathlib import Path
import json

for mims_image in MIMSImage.objects.all():
  mims_path = Path(mims_image.file.path)
  reg_loc = Path(
        mims_path.parent / mims_path.stem / "registration" / "reg_shapes.json"
    )
  if reg_loc.exists() and not mims_image.registration_info:
    print(mims_image.file.name, "exists")
    with open(reg_loc, "r") as f:
      reg_shapes = json.load(f)
      mims_image.registration_info = {
          "em_shapes": reg_shapes.get("em_shapes", []),
          "mims_shapes": reg_shapes.get("mims_shapes", []),
          "em_points": reg_shapes.get("em_points", []),
          "mims_points": reg_shapes.get("mims_points", []),
      }
    mims_image.save()
  elif not reg_loc.exists():
    print("----------", mims_image.file.name, "doesn't exist")

---------- mims_image_sets/a49c7172-8f55-4960-9752-8b669b20014d/mims_images/73_ROI_5.im doesn't exist
---------- mims_image_sets/a49c7172-8f55-4960-9752-8b669b20014d/mims_images/73_ROI_6.im doesn't exist
---------- mims_image_sets/a49c7172-8f55-4960-9752-8b669b20014d/mims_images/73_ROI_7.im doesn't exist
---------- mims_image_sets/a49c7172-8f55-4960-9752-8b669b20014d/mims_images/73_ROI_8.im doesn't exist
---------- mims_image_sets/a1b5db3c-b189-4bb1-8d0c-c4311a2b9b2a/mims_images/79_mosaic7_4.im doesn't exist
---------- mims_image_sets/a1b5db3c-b189-4bb1-8d0c-c4311a2b9b2a/mims_images/79_mosaic7_5.im doesn't exist
---------- mims_image_sets/a1b5db3c-b189-4bb1-8d0c-c4311a2b9b2a/mims_images/79_mosaic7_14.im doesn't exist
---------- mims_image_sets/5a2ec271-ff81-40a9-9e62-2fb520ec441c/mims_images/77_mosaic1_6.im doesn't exist
---------- mims_image_sets/a4a35f29-9339-40b3-8df6-7fed4c754776/mims_images/81_mosaic_5.im doesn't exist
---------- mims_image_sets/a4a35f29-9339-40b3-8df6-7fed4c75477

In [2]:
from mims.services.register import register_images

canvas_names = ["73_6hrfast_M1", "75_6hrfast_M2", "77_6hrfast_M3", "79_1hr13C_M1",
                "81_1hr13C_M2_1", "81_1hr13C_M2_2", "83_1hr13C_M3", "85_ONfast_M1",
                "87_ONfast_M2", "89_ONfast_M3"]
#canvas_names = ["79_1hr13C_M1"]
mims_to_register = None
for canvas_name in canvas_names:
    canvas = Canvas.objects.get(name=canvas_name)
    mims_images = MIMSImage.objects.filter(canvas=canvas)
    # Get the counts for each status and print with canvas name
    statuses = list(set([m.status for m in mims_images]))
    status_counts = {status: len([m for m in mims_images if m.status == status]) for status in statuses}
    registering = [m for m in mims_images]
    
    for mims_image in registering:
        mims_to_register = mims_image
        print("registering", mims_image.file.name)
        register_images(mims_image.id)

    
    

registering mims_image_sets/937c681b-80c9-4b8e-b69c-30cb8787e650/mims_images/73_mosaic_1.im
/Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/937c681b-80c9-4b8e-b69c-30cb8787e650/mims_images/final/14N_12C_unwarped_QMa746k.png
/Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/937c681b-80c9-4b8e-b69c-30cb8787e650/mims_images/final/197Au_unwarped_9N0p8s1.png
/Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/937c681b-80c9-4b8e-b69c-30cb8787e650/mims_images/final/31P_unwarped_yk6pRvt.png
/Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/937c681b-80c9-4b8e-b69c-30cb8787e650/mims_images/final/12C_unwarped_FW3gMx1.png
/Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/937c681b-80c9-4b8e-b69c-30cb8787e650/mims_images/final/SE_unwarped_ZO0cL32.png
/Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/937c681b-80c9-4b8e-b69c-30cb8787e650/mims_images/fina